### Get parameters

In [ ]:
acre = get_inputs("acre")
improvement = get_inputs("improvement").split(",")
deed = get_inputs("deed")
owner_name = get_inputs("owner_name")
address = get_inputs("address").split(",")

### Read in data

In [ ]:
# Make sure address columns are read in as string

dict_dtypes = {x: "str" for x in address}

In [ ]:
df_raw = pd.read_csv(
    f"../../../Dropbox/Raw Data/{county}_id_raw.csv", dtype=dict_dtypes, low_memory=False
)
df_raw.reset_index(inplace=True)

In [ ]:
# Make a record id column

df_raw["record_id"] = county + "_" + df_raw["index"].astype("str")
df_raw.drop(["index"], axis=1, inplace=True)

In [ ]:
# Create a copy of the raw file

df = df_raw.copy(deep=True)

### Start functions

In [ ]:
# Keep any land with 0 acres or greater than or equal to 0.06 acres

df = filter_acres(df, acre)

In [ ]:
df = flag_owner_with_improvements(df, improvement, owner_name)

In [ ]:
# For now, remove the people who had any property with improvements

# df = df[df.owner_with_improvement != 1].reset_index(drop=True)

In [ ]:
# Remove all rows associated with improvements, just keep the market value of 0

df = df[df[improvement].sum(axis=1) == 0].reset_index(drop=True)

#### Fix owner name


In [ ]:
df = remove_owner_name_blank(df, owner_name)

In [ ]:
df = flag_trust_co_columns(df, owner_name, address, trust_filter, co_filter)

In [ ]:
before = len(df)

df = clean_trust_co_columns(df, co_filter, owner_name, address)

after = len(df)

print(
    f"co/trustee function working for {county}"
    if before == after
    else f"error in co/trustee function for {county}"
)

In [ ]:
# Get the final address combining both the original address/name columns with the adjusted columns

for i in address:

    df[i + "_final"] = df[i + "_adj"].fillna(df[i])

    # Specifically remove the values of "Remove" and replace with np.nan

    df[i + "_final"] = np.where(df[i + "_final"] == "Remove", np.nan, df[i + "_final"])

df[owner_name + "_final"] = df[owner_name + "_adj"].fillna(df[owner_name])

In [ ]:
# Get list of columns for the adjusted ownername and addresses

owner_name_final = owner_name + "_final"
address_final = [x + "_final" for x in address]

In [ ]:
# Use original owner name column to filter down the list of rows that we want

df = remove_owner_name_filter_items(df, owner_name, owner_name_final, owner_name_filter)

#### Clean data - Deed column

In [ ]:
df = clean_deed(df, deed)

#### Create distinct address to name mapping

- Choose one name per address

In [ ]:
# Create address to process

df = create_address_to_process(df, address_final)

# Create address combined for raw file for reference

df["raw_address_combined"] = df[address].fillna("").agg(", ".join, axis=1)

In [ ]:
# Create an address to name mapping table

address_owner_name_table_ = address_owner_name_table(df, owner_name_final)

In [ ]:
# Merge mapped chosen owner name back to the original dataset
# --> group by address, choose one owner only by deed year (x)

df = merge_chosen_owner_name(df, address_owner_name_table_)

#### Choose most recent address for an owner name

- Get most recent address for an owner name

In [ ]:
df = most_recent_address_table(df)

#### Create a table of number of properties owned by owner name so we can exclude the ones over 7

In [ ]:
df = filter_parcels(df)

#### Format names

In [ ]:
df = clean_owner_name(df)

#### Format addresses

In [ ]:
df = clean_address_to_process(df)

#### Create flags for potential name/address issues for manual review

In [ ]:
df = address_issue(df)

In [ ]:
df = owner_name_issue(df)

In [ ]:
df = trustee_co_issue(df)

In [ ]:
# Get total number of issues

df["n_issues"] = df["address_issue"] + df["owner_name_issue"] + df["trustee_co_issue"]

#### Export data

In [ ]:
# Mailing list

df[df.mailing_address_flag == 1][
    [
        "chosen_record_id",
        owner_name,
        "chosen_owner_name",
        "chosen_owner_name_to_process",
        "raw_address_combined",
        "address_to_process",
        "chosen_owner_with_improvement",
        "n_parcels",
        "full_name",
        "first_name",
        "last_name",
        "address",
        "city",
        "state",
        "zip",
        "address_issue",
        "owner_name_issue",
        "trustee_co_issue",
        "n_issues",
    ]
].sort_values(
    ["n_issues", "address_issue", "owner_name_issue", "trustee_co_issue"],
    ascending=False,
).reset_index(
    drop=True
).to_csv(
    f"../../../Dropbox/Files/{county}/mailing_list.csv", index=False
)

In [ ]:
# Raw data, cleaned to included rows with cleaning columns and ids

df.sort_values(
    ["n_issues", "address_issue", "owner_name_issue", "trustee_co_issue"],
    ascending=False,
).reset_index(drop=True).to_csv(
    f"../../../Dropbox/Files/{county}/raw_cleaned_with_id.csv",
    index=False,
)

In [ ]:
# Mailing list owners, merged with raw file

df_raw.merge(df[["record_id", "chosen_record_id"]], on="record_id", how="left").to_csv(
    f"../../../Dropbox/Files/{county}/raw_with_id.csv", index=False
)